# Compare building height and population

1. Stack population and builidng heights
2. Create 2o population layer describing 3x3 population sum
3. Calculate service access

TODO: Add metadata

1. DLR height
2. WP Population


In [1]:
import sys, os, importlib, shutil
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../../GOST/")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc
import GOSTRocks.osmMisc as osm_misc

import hot_spot_mapping as hot

In [2]:
wp_vrt = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/WP_2020_100m.vrt"
wp = rasterio.open(wp_vrt)
globcover_data = "/home/public/Data/GLOBAL/LANDCOVER/GLOBCOVER/2015/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
inWSF_folder = "/home/public/Data/PROJECTS/LEI/DLR_V2"
out_folder = "/home/wb411133/data/Projects/HeightComparison"

#Get a list of cities
allCities = {}
for root, dirs, files in os.walk(inWSF_folder):
    for f in files:
        if f[-4:] == ".tif":
            cityName = f.split("_")[0] # .replace(".tif", "")#
            if not cityName in allCities.keys():
                allCities[cityName] = [os.path.join(root, f)]
            else:
                allCities[cityName].append(os.path.join(root, f))
                


In [3]:
allCities

{'Zwolle': ['/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSF3D_AW3D30.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSF3D_AW3D30_nScenes.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSF3D_AW3D30_RGB.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSFevolution.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSFevolution_IDCscore.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Zwolle_WSFevolution_RGB.tif'],
 'Aalborg': ['/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSF3D_AW3D30.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSF3D_AW3D30_nScenes.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSF3D_AW3D30_RGB.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSFevolution.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSFevolution_IDCscore.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2/Aalborg_WSFevolution_RGB.tif'],
 'Abadan': ['/home/public/Data/PROJECTS/LEI/DLR_V2/Abadan_WSF3D_AW3D30.tif',
  '/home/public/Data/PROJECTS/LEI/DLR_V2

In [4]:
glob_pop_raster = rasterio.open(wp_vrt)
globR = rasterio.open(globcover_data)


In [5]:
city_name = "Dhaka"
cur_folder = os.path.join(out_folder, city_name) 
city = hot.city_hotspot(allCities[city_name][0], cur_folder)
pop_hotspots = os.path.join(cur_folder, "pop_hotspots.tif")
city.calculate_pop_hotspots(out_file = pop_hotspots)
        

/home/wb411133/Code/CityScan/VulnerabilityMapping/hot_spot_mapping.py:350: RuntimeWarning: divide by zero encountered in true_divide
  pop_by_floor = pop_data/height_data
/home/wb411133/Code/CityScan/VulnerabilityMapping/hot_spot_mapping.py:350: RuntimeWarning: invalid value encountered in true_divide
  pop_by_floor = pop_data/height_data
/home/wb411133/Code/CityScan/VulnerabilityMapping/hot_spot_mapping.py:351: RuntimeWarning: invalid value encountered in less
  pop_by_floor[pop_by_floor < 0] = 0
/home/wb411133/Code/CityScan/VulnerabilityMapping/hot_spot_mapping.py:352: RuntimeWarning: invalid value encountered in greater
  pop_by_floor[pop_by_floor > 10000] = 0


In [ ]:
importlib.reload(hot)

all_results = []
base_map = "/home/wb411133/data/Projects/HeightComparison/Dhaka/map.qgz" 
for city_name in allCities.keys():
    cur_folder = os.path.join(out_folder, city_name) 
    if True:#not city_name in res['city'].values:
        #try:
        misc.tPrint("Processing %s" % city_name)
        if not os.path.exists(cur_folder):
            os.makedirs(cur_folder)
        city = hot.city_hotspot(allCities[city_name][0], cur_folder)
        city.extract_other_rasters(glob_pop_raster, globR)
        rosads = city.extract_osm_data()
        xx = city.generate_walking_raster()
        city.calculate_accessibility()
        city.calculate_pop_hotspots()
        accessibility_hotspots = []
        for time_raster in [os.path.join(cur_folder, "shops_distance_roads.tif"),
                            os.path.join(cur_folder, "toilets_distance_roads.tif"),
                            os.path.join(cur_folder, "water_points_distance_roads.tif")]:
            out_file = os.path.join(cur_folder, time_raster.replace("distance_roads", "hotspots"))
            accessibility_hotspots.append(out_file)
            if (os.path.exists(time_raster)) and (not os.path.exists(out_file)):
                city.calculate_accessibility_hotspots(time_raster, out_file)
        results = city.combine_results(base_map, accessibility_hotspots)
        results['city'] = city_name
        all_results.append(results)
        #except:
        #    misc.tPrint("Error processing %s" % city_name)

In [ ]:
res = pd.DataFrame(all_results)

In [ ]:
res.to_csv("%s_Hotspot_summary_2.csv" % inWSF_folder)

In [ ]:
inWSF_folder

# Zip results folders

In [ ]:
for f in os.listdir(out_folder):
    cur_folder = os.path.join(out_folder, f)
    if os.path.isdir(cur_folder):
        out_file = os.path.join(out_folder, f"{f}.zip")
        if not os.path.exists(out_file):
            print(f'zip -r {f}.zip {f}')
        

# Debugging Below

In [ ]:
importlib.reload(hot)
city_name="Dhaka"
cur_folder = os.path.join(out_folder, city_name)
time_raster = os.path.join(cur_folder, "shops_distance_roads.tif")

base_map = "/home/wb411133/data/Projects/HeightComparison/Dhaka/map.qgz" 


city = hot.city_hotspot(allCities[city_name][0], cur_folder)
city.extract_other_rasters(glob_pop_raster, globR)
rosads = city.extract_osm_data()
xx = city.generate_walking_raster()
city.calculate_accessibility()
city.calculate_pop_hotspots()
accessibility_hotspots = []
for time_raster in [os.path.join(cur_folder, "shops_distance_roads.tif"),
                    os.path.join(cur_folder, "toilets_distance_roads.tif"),
                    os.path.join(cur_folder, "water_points_distance_roads.tif")]:
    out_file = os.path.join(cur_folder, time_raster.replace("distance_roads", "hotspots"))
    accessibility_hotspots.append(out_file)
    city.calculate_accessibility_hotspots(time_raster, out_file)
results = city.combine_results(basemap, accessibility_hotspots)
results['city'] = city_name
            